In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, accuracy_score,precision_score,recall_score, matthews_corrcoef
import pickle

In [2]:
def BasePred(X_test,y_test):
  #thresholds and base models performance
  label = [[0, 1][i <= 500] for i in X_test['ann']]
  print("Ann:\nAcc: ",metrics.accuracy_score(label,y_test))
  print('Recall:',metrics.recall_score(y_test, label))
  print('Percision:',metrics.precision_score(y_test, label))
  print('F1:',metrics.f1_score(y_test, label))
  print('------------------------------------------')
  label = [[0, 1][r <= 2] for r in X_test['consensus']]
  print("consensus:\nAcc: ", metrics.accuracy_score(label,y_test))
  print('Recall:', metrics.recall_score(y_test, label))
  print('Percision:', metrics.precision_score(y_test, label))
  print('F1:', metrics.f1_score(y_test, label))
  print('------------------------------------------')
  label = [[0, 1][i <= 500] for i in X_test['netmhcpan_ba']]
  print("netmhcpan_ba:\nAcc:  ",metrics.accuracy_score(label,y_test))
  print('Recall:',metrics.recall_score(y_test, label))
  print('Percision:',metrics.precision_score(y_test, label))
  print('F1:',metrics.f1_score(y_test, label))
  print('------------------------------------------')
  label = [[0, 1][score >= 0.5] for score in X_test['netmhcpan_el']]
  print("netmhcpan_el:\nAcc:  ",metrics.accuracy_score(label,y_test))
  print('Recall:',metrics.recall_score(y_test, label))
  print('Percision:',metrics.precision_score(y_test, label))
  print('F1:',metrics.f1_score(y_test, label))
  print('------------------------------------------')
  label = [[0, 1][i <= 500] for i in X_test['smm']]
  print("smm:\nAcc:  ",metrics.accuracy_score(label,y_test))
  print('Recall:',metrics.recall_score(y_test, label))
  print('Percision:',metrics.precision_score(y_test, label))
  print('F1:',metrics.f1_score(y_test, label))
  print('------------------------------------------')
  label = [[0, 1][i <= 500] for i in X_test['smmpmbec']]
  print("smmpmbec:\nAcc:  ",metrics.accuracy_score(label,y_test))
  print('Recall:',metrics.recall_score(y_test, label))
  print('Percision:',metrics.precision_score(y_test, label))
  print('F1:',metrics.f1_score(y_test, label))
  print('------------------------------------------')
  label = [[0, 1][i <= 500] for i in X_test['pickpocket']]
  print("pickpocket:\nAcc:  ",metrics.accuracy_score(label,y_test))
  print('Recall:',metrics.recall_score(y_test, label))
  print('Percision:',metrics.precision_score(y_test, label))
  print('F1:',metrics.f1_score(y_test, label))
  print('------------------------------------------')
  label = [[0, 1][i <= 500] for i in X_test['netmhccons']]
  print("netmhccons:\nAcc:  ",metrics.accuracy_score(label,y_test))
  print('Recall:',metrics.recall_score(y_test, label))
  print('Percision:',metrics.precision_score(y_test, label))
  print('F1:',metrics.f1_score(y_test, label))
  print('------------------------------------------')
  label = [[0, 1][score >= 0.5] for score in X_test['netmhcstabpan']]
  print("netmhcstabpan:\nAcc:  ",metrics.accuracy_score(label,y_test))
  print('Recall:',metrics.recall_score(y_test, label))
  print('Percision:',metrics.precision_score(y_test, label))
  print('F1:',metrics.f1_score(y_test, label))
  print('------------------------------------------')

##Evaluate the model on validation data

In [17]:
X_test = pd.read_csv('ESM_test.csv').drop(columns=['peptide','HLA','Unnamed: 0'])
y_test = pd.read_csv('label.csv').drop(columns=['Unnamed: 0'])

In [18]:
#Other models
BasePred(X_test,y_test)

Ann:
Acc:  0.8029618271325962
Recall: 0.626408760290944
Percision: 0.9740243599304002
F1: 0.7624653402733862
------------------------------------------
consensus:
Acc:  0.7996529739326931
Recall: 0.619934457677244
Percision: 0.9736379613356766
F1: 0.757532841724862
------------------------------------------
netmhcpan_ba:
Acc:   0.8309256718586071
Recall: 0.6810806490288546
Percision: 0.9770668501318656
F1: 0.8026563677467973
------------------------------------------
netmhcpan_el:
Acc:   0.8014688080058107
Recall: 0.6140196626968268
Percision: 0.98829280843947
F1: 0.7574442910668507
------------------------------------------
smm:
Acc:   0.7891211363086111
Recall: 0.6391975061945488
Percision: 0.9182454931679871
F1: 0.7537229029217719
------------------------------------------
smmpmbec:
Acc:   0.787264950367202
Recall: 0.6316041883142834
Percision: 0.9225919439579685
F1: 0.7498576579996205
------------------------------------------
pickpocket:
Acc:   0.7146315874424986
Recall: 0.4479258

In [19]:
# load the model
model = pickle.load(open('SiaScoreNet.pkl', 'rb'))

In [20]:
#SiaScoreNet
X_test_ens =  X_test.iloc[:,:9].values
X_test_peptide = X_test.iloc[:,9:329].values
X_test_mhc = X_test.iloc[:,329:].values

val_loss, val_accuracy = model.evaluate([X_test_peptide, X_test_mhc, X_test_ens], y_test)
print(f'Validation accuracy: {val_accuracy:.4f}')
y_pred = model.predict([X_test_peptide, X_test_mhc, X_test_ens])
y_pred = y_pred.flatten()
y_pred_c = np.where(y_pred > 0.5, 1, 0)

test_roc_auc = roc_auc_score(y_test, y_pred)
test_au_pr = average_precision_score(y_test, y_pred)
test_mcc = matthews_corrcoef(y_test, y_pred_c)
test_f1 = f1_score(y_test, y_pred_c)
test_acc = accuracy_score(y_test, y_pred_c)

print(f'Acc: {test_acc},  \nF1: {test_f1}')
print('Recall:',recall_score(y_test, y_pred_c))
print('Percision:',precision_score(y_test, y_pred_c))

775/775 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9188 - loss: 0.2217
Validation accuracy: 0.9216
775/775 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
Acc: 0.9215963199096118,  
F1: 0.9214600428473261
Recall: 0.911038286308049
Percision: 0.932122996401701
